# K-means and K-means++

参考资料：
- [原创 | 一文读懂K均值（K-Means）聚类算法](https://mp.weixin.qq.com/s/XS1_NAZ65P-1aLqwWm2hbg)
- [K-Means(K均值聚类算法) - 范永康的文章 - 知乎](https://zhuanlan.zhihu.com/p/136842949)

## K-means

首先导入一些必要的库。这里仅使用python内置的库来实现两种不同的算法。

In [56]:
import math
import pprint
import random

接下来构建一些关键的辅助函数。由于K-means是一种基于样本与聚类中心的相似度的方法，所以度量两个元素之间的相似关系的函数是很重要的。这里不应该对样本的特征数进行约束。我们选择欧式距离作为距离度量。

In [57]:
def cal_distance(pt1, pt2):
    return math.sqrt(sum([(x-y)**2 for x, y in zip(pt1, pt2)]))

除了计算距离之外，另一个会被反复使用的功能就是计算质心了。我们选择所有样本的特征向量的均值作为质心的表示。

In [58]:
def cal_centroid(pts):
    num_pts = len(pts)
    num_features = len(pts[0])
    centroid = [0] * num_features
    for pt in pts:
        for i in range(num_features):
            centroid[i] += pt[i] / num_pts
    return centroid

设定聚类中心的数量。

In [59]:
K = 3

接下来就要开始编写K-means的核心代码了。我们需要了解一下K-means的核心步骤：
1. 初始化聚类中心
2. 遍历各个样本，将各自分配到最近的聚类中心
3. 对各个聚类中心对应的样本集合计算质心作为新的聚类中心
4. 如果3中聚类中心有变化，则进入2，否则结束更新，输出聚类结果。

In [60]:
def initialize_centroids(pts: dict, k: int) -> list:
    centroids = random.sample([v['value'] for v in pts.values()], k=k)
    return centroids

In [61]:
def classify_samples(pts: dict, centroids: list):
    # 根据执行计算各点与当前质心的距离，并按照最短距离判定各点属于哪个质心
    for pt_name, pt_info in pts.items():
        pt_value = pt_info["value"]
        if pt_value in centroids:
            pt_info["centroid"] = pt_value
            continue

        min_dist_pt_to_centroids = None
        pt_centroid_value = None
        for centroid_value in centroids:
            dist = cal_distance(pt_value, centroid_value)
            if pt_centroid_value is None or dist < min_dist_pt_to_centroids:
                min_dist_pt_to_centroids = dist
                pt_centroid_value = centroid_value
        pt_info["centroid"] = pt_centroid_value

In [62]:
def update_centroids(pts: dict, centroids: list) -> list:
    new_centroids = []
    for centroid_value in centroids:
        pt_group = []
        for pt_name, pt_info in pts.items():
            if pt_info["centroid"] == centroid_value:
                pt_group.append(pt_info["value"])            
        new_centroid = cal_centroid(pt_group)
        new_centroids.append(new_centroid)
        for pt_name, pt_info in pts.items():
            if pt_info["centroid"] == centroid_value:
                pt_info["centroid"] = new_centroid
    return new_centroids

基于这些核心的函数，我们可以得到最终的k-means函数。

In [63]:
def k_means(pts: dict, k: int):
    centroids = initialize_centroids(pts=pts, k=k)
    print("Initial Centroids: ", centroids)

    iterations = 0
    while True:
        print("Start Iteration ", iterations)
        classify_samples(pts=pts, centroids=centroids)
        new_centroids = update_centroids(pts=pts, centroids=centroids)
        total_diff = sum([cal_distance(x, y) for x, y in zip(centroids, new_centroids)])
        if total_diff == 0:
            break
        centroids = new_centroids
        print("After Iteration ", iterations)
        pprint.pprint(pts)
        
        iterations += 1
    print("End Iteration")

In [64]:
pts = dict(
    a=dict(centroid=None, value=(-5.379713, -3.362104)),
    b=dict(centroid=None, value=(-3.487105, -1.724432)),
    c=dict(centroid=None, value=(0.450614, -3.302219)),
    d=dict(centroid=None, value=(-0.392370, -3.963704)),
    e=dict(centroid=None, value=(-3.453687, 3.424321)),
)
pprint.pprint(pts)

{'a': {'centroid': None, 'value': (-5.379713, -3.362104)},
 'b': {'centroid': None, 'value': (-3.487105, -1.724432)},
 'c': {'centroid': None, 'value': (0.450614, -3.302219)},
 'd': {'centroid': None, 'value': (-0.39237, -3.963704)},
 'e': {'centroid': None, 'value': (-3.453687, 3.424321)}}


In [65]:
# random.seed(0)
k_means(pts=pts, k=K)
pprint.pprint(pts)

Initial Centroids:  [(0.450614, -3.302219), (-0.39237, -3.963704), (-3.487105, -1.724432)]
Start Iteration  0
After Iteration  0
{'a': {'centroid': [-4.106835, -0.5540716666666665],
       'value': (-5.379713, -3.362104)},
 'b': {'centroid': [-4.106835, -0.5540716666666665],
       'value': (-3.487105, -1.724432)},
 'c': {'centroid': [0.450614, -3.302219], 'value': (0.450614, -3.302219)},
 'd': {'centroid': [-0.39237, -3.963704], 'value': (-0.39237, -3.963704)},
 'e': {'centroid': [-4.106835, -0.5540716666666665],
       'value': (-3.453687, 3.424321)}}
Start Iteration  1
End Iteration
{'a': {'centroid': [-4.106835, -0.5540716666666665],
       'value': (-5.379713, -3.362104)},
 'b': {'centroid': [-4.106835, -0.5540716666666665],
       'value': (-3.487105, -1.724432)},
 'c': {'centroid': [0.450614, -3.302219], 'value': (0.450614, -3.302219)},
 'd': {'centroid': [-0.39237, -3.963704], 'value': (-0.39237, -3.963704)},
 'e': {'centroid': [-4.106835, -0.5540716666666665],
       'value': 

## K-means++

K-means的聚类效果和初始聚类中心的选择有着很大的关系。K-means++则是对这一关键步骤的优化。即初始质心的选择应该尽可能的分散。

In [66]:
def min_dist_with_centroids(pt: tuple, centroids: list) -> float:
    # 计算样本与最近的质心的距离
    all_dists = [cal_distance(pt, c) for c in centroids]
    return min(all_dists)

这里根据各点对应的最小距离确定点采样的概率。
如何根据权重来确定概率，实现这点的算法有很多，其中比较简单的是轮盘法。
轮盘赌法：即将所有样本各自对应的权重值等价为轮盘区块的面积，并使用一个0~1均匀采样获得最终的概率采样。
这一过程就类似于轮盘指针的旋转。这里的实现实际上是按照累计概率的方式进行的。

In [67]:
def initialize_centroids_plusplus(pts, k):
    pt_names = list(pts.keys())
    centroids = [random.choice([pt["value"] for pt in pts.values()])]
    # centroids = [pts["d"]["value"]]

    for i in range(K - 1):
        total_dists = []
        for pt_name in pt_names:
            pt_info = pts[pt_name]  # 这里无需考虑样本是否为质心的情况，因为此时的最小距离为0，在轮盘赌的时候其并不会起到影响
            min_dist = min_dist_with_centroids(pt=pt_info["value"], centroids=centroids)
            total_dists.append(min_dist)

        # 轮盘赌算法
        sample_dist = sum(total_dists) * random.random()
        for i, dist in enumerate(total_dists):
            sample_dist -= dist
            if sample_dist <= 0:
                centroid_name = pt_names[i]
                centroid_value = pts[centroid_name]["value"]
                break
        centroids.append(centroid_value)
    return centroids

In [68]:
def k_means_pp(pts: dict, initial_centroids: list):
    centroids = initial_centroids
    print("Initial Centroids: ", centroids)

    iterations = 0
    while True:
        print("Start Iteration ", iterations)
        classify_samples(pts=pts, centroids=centroids)
        new_centroids = update_centroids(pts=pts, centroids=centroids)
        total_diff = sum([cal_distance(x, y) for x, y in zip(centroids, new_centroids)])
        if total_diff == 0:
            break
        centroids = new_centroids
        print("After Iteration ", iterations)
        pprint.pprint(pts)

        iterations += 1
    print("End All Iterations")

In [69]:
pts = dict(
    a=dict(centroid=None, value=(-5.379713, -3.362104)),
    b=dict(centroid=None, value=(-3.487105, -1.724432)),
    c=dict(centroid=None, value=(0.450614, -3.302219)),
    d=dict(centroid=None, value=(-0.392370, -3.963704)),
    e=dict(centroid=None, value=(-3.453687, 3.424321)),
)
pprint.pprint(pts)

{'a': {'centroid': None, 'value': (-5.379713, -3.362104)},
 'b': {'centroid': None, 'value': (-3.487105, -1.724432)},
 'c': {'centroid': None, 'value': (0.450614, -3.302219)},
 'd': {'centroid': None, 'value': (-0.39237, -3.963704)},
 'e': {'centroid': None, 'value': (-3.453687, 3.424321)}}


In [70]:
random.seed(0)
initial_centroids = initialize_centroids_plusplus(pts=pts, k=K)
k_means_pp(pts=pts, initial_centroids=initial_centroids)
pprint.pprint(pts)

Initial Centroids:  [(-0.39237, -3.963704), (-3.453687, 3.424321), (-5.379713, -3.362104)]
Start Iteration  0
After Iteration  0
{'a': {'centroid': [-4.433409, -2.543268], 'value': (-5.379713, -3.362104)},
 'b': {'centroid': [-4.433409, -2.543268], 'value': (-3.487105, -1.724432)},
 'c': {'centroid': [0.02912200000000001, -3.6329615],
       'value': (0.450614, -3.302219)},
 'd': {'centroid': [0.02912200000000001, -3.6329615],
       'value': (-0.39237, -3.963704)},
 'e': {'centroid': [-3.453687, 3.424321], 'value': (-3.453687, 3.424321)}}
Start Iteration  1
End All Iterations
{'a': {'centroid': [-4.433409, -2.543268], 'value': (-5.379713, -3.362104)},
 'b': {'centroid': [-4.433409, -2.543268], 'value': (-3.487105, -1.724432)},
 'c': {'centroid': [0.02912200000000001, -3.6329615],
       'value': (0.450614, -3.302219)},
 'd': {'centroid': [0.02912200000000001, -3.6329615],
       'value': (-0.39237, -3.963704)},
 'e': {'centroid': [-3.453687, 3.424321], 'value': (-3.453687, 3.424321)}}